In [1]:
import numpy as np
import mujoco
from mujoco_sysid import regressors
from mujoco_sysid.utils import muj2pin
import pinocchio as pin
from mujoco_sysid.parameters import get_dynamic_parameters

np.random.seed(0)

In [2]:
from robot_descriptions.skydio_x2_description import URDF_PATH
from robot_descriptions.skydio_x2_mj_description import MJCF_PATH

In [3]:
pinmodel = pin.buildModelFromUrdf(URDF_PATH)
pindata = pinmodel.createData()

mjmodel = mujoco.MjModel.from_xml_path(MJCF_PATH)
mjmodel.opt.gravity = np.zeros(3)
for act_id in range(4):
    mjmodel.actuator(act_id).ctrlrange = np.array([-1e4, 1e4])
mjdata = mujoco.MjData(mjmodel)
theta = get_dynamic_parameters(mjmodel, 1)

In [4]:
q = np.random.rand(pinmodel.nq)
# normalize the quaternion
q[3:7] /= np.linalg.norm(q[3:7])
# q = np.array([0, 0, 1, 1, 0, 0, 0])
v, dv = np.random.rand(pinmodel.nv), np.random.rand(pinmodel.nv)

# v[:3] *= 0  # FIXME: occasionally when we set the linear velocity to zero, the test works
# otherwise the inverse dynamics does not match

pinq, pinv = muj2pin(q, v)

print("Configuration")
print(f"Pinocchio: {pinq}, {pinv}, {dv}")
print(f"Mujoco: {q}, {v}, {dv}")

# Selector matrix for actuators
selector = np.array(
    [
        [0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0],
        [1.0, 1.0, 1.0, 1.0],
        [-0.18, 0.18, 0.18, -0.18],
        [0.14, 0.14, -0.14, -0.14],
        [-0.0201, 0.0201, 0.0201, -0.0201],
    ]
)

# Random control input
ctrl = np.zeros(4)  # np.random.randn(4)

# compute with pinocchio all terms
tau = selector @ ctrl
dv = pin.aba(pinmodel, pindata, pinq, pinv, tau)
result = pin.rnea(pinmodel, pindata, pinq, pinv, dv)

# compute with mujoco all terms
mjdata.qpos[:] = q
mjdata.qvel[:] = v
mjdata.qacc[:] = dv
mjdata.ctrl[:] = ctrl

mujoco.mj_step(mjmodel, mjdata)
# mujoco.mj_inverse(mjmodel, mjdata)

Configuration
Pinocchio: [0.5488135  0.71518937 0.60276338 0.40671359 0.62006595 0.42008887
 0.52309427], [0.68314008 0.72550602 0.93695947 0.79172504 0.52889492 0.56804456], [0.92559664 0.07103606 0.0871293  0.0202184  0.83261985 0.77815675]
Mujoco: [0.5488135  0.71518937 0.60276338 0.52309427 0.40671359 0.62006595
 0.42008887], [0.891773   0.96366276 0.38344152 0.79172504 0.52889492 0.56804456], [0.92559664 0.07103606 0.0871293  0.0202184  0.83261985 0.77815675]


In [5]:
pinY = pin.jointBodyRegressor(pinmodel, pindata, 1)

with np.printoptions(precision=5, suppress=True, linewidth=400):
    print(pinY)

[[-0.04571 -0.6024   0.36044  0.84716  0.       0.       0.       0.       0.       0.     ]
 [-0.03027  0.47704 -0.9495   0.56095  0.       0.       0.       0.       0.       0.     ]
 [ 0.04892  0.05231  0.03992 -0.90656  0.       0.       0.       0.       0.       0.     ]
 [ 0.       0.       0.04892  0.03027 -0.26052 -0.05231 -0.30044  0.47704 -0.04294  0.30044]
 [ 0.      -0.04892  0.      -0.04571  0.44974  0.03992  0.39743 -0.30415 -0.36044 -0.44974]
 [ 0.      -0.03027  0.04571  0.      -0.41874  0.3471   0.41874 -0.56095  0.84716  0.0583 ]]


In [6]:
# pin.forwardKinematics(pinmodel, pindata, pinq, pinv)
# pin.computeAllTerms(pinmodel, pindata, pinq, pinv)
# pin.aba(pinmodel, pindata, pinq, pinv, tau)
# pin.getVelocity(pinmodel, pindata, 1), pin.getClassicalAcceleration(pinmodel, pindata, 1)

pindata.a_gf[1]

  v = -0.129148   0.32349 -0.164172
  w = -0.260518  0.397427 0.0582974

In [7]:
pindata.a_gf[1]

  v = -0.129148   0.32349 -0.164172
  w = -0.260518  0.397427 0.0582974

In [8]:
pindata.liMi[1].actInv(pindata.a_gf[0])

  v =  -3.01161   9.28482 -0.978993
  w =  0 -0  0

In [9]:
pinq

array([0.5488135 , 0.71518937, 0.60276338, 0.40671359, 0.62006595,
       0.42008887, 0.52309427])

In [10]:
mjdata.xmat[1].reshape(3, 3)

array([[-0.12191288,  0.06488634,  0.99041759],
       [ 0.94387066,  0.31621879,  0.09546651],
       [-0.30699418,  0.94646471, -0.09979545]])

In [11]:
pindata.oMi[1]

  R =
1 0 0
0 1 0
0 0 1
  p = 0 0 0

In [12]:
pindata.liMi[1]

  R =
 -0.121913  0.0648863   0.990418
  0.943871   0.316219  0.0954665
 -0.306994   0.946465 -0.0997955
  p = 0.548814 0.715189 0.602763

In [13]:
# compute with mujoco all terms
mjdata.qpos[:] = q
mjdata.qvel[:] = v
mjdata.qacc[:] = dv
mjdata.ctrl[:] = ctrl

# position
mujoco.mj_kinematics(mjmodel, mjdata)
mujoco.mj_comPos(mjmodel, mjdata)
mujoco.mj_crb(mjmodel, mjdata)
mujoco.mj_factorM(mjmodel, mjdata)

# velocity
mujoco.mj_fwdVelocity(mjmodel, mjdata)
mujoco.mj_comVel(mjmodel, mjdata)
mujoco.mj_referenceConstraint(mjmodel, mjdata)
# mujoco.mj_rne(mjmodel, mjdata)

mujoco.mj_fwdActuation(mjmodel, mjdata)
mujoco.mj_fwdAcceleration(mjmodel, mjdata)

mjdata.qacc_smooth

array([-0.04280117, -0.06304948, -0.01692062, -0.27890477,  0.39128987,
        0.05578023])

In [14]:
accel = np.zeros(6)

mujoco.mj_rnePostConstraint(mjmodel, mjdata)
mujoco.mj_objectAcceleration(mjmodel, mjdata, 2, 1, accel, 1)

accel

array([-0.2605178 ,  0.39742672,  0.05829744, -9.05989038, -1.87550827,
        1.91802153])

In [15]:
pindata.a_gf[1]

  v = -0.129148   0.32349 -0.164172
  w = -0.260518  0.397427 0.0582974

In [16]:
velocity = np.zeros(6)

mujoco.mj_objectVelocity(mjmodel, mjdata, 2, 1, velocity, 1)

v, w = velocity[3:], velocity[:3]

# accel[3:] -= np.cross(w, v)

accel

array([-0.2605178 ,  0.39742672,  0.05829744, -9.05989038, -1.87550827,
        1.91802153])

In [17]:
# try to reverse all the values
pindata.a_gf[1] - pindata.liMi[1].actInv(pindata.a_gf[0])

  v =  2.88246 -8.96133 0.814821
  w = -0.260518  0.397427 0.0582974

In [18]:
pindata.v[1]

  v =  0.68314 0.725506 0.936959
  w = 0.791725 0.528895 0.568045

In [19]:
# check that angular part matches
accel[:3], pindata.a_gf[1].angular

(array([-0.2605178 ,  0.39742672,  0.05829744]),
 array([-0.2605178 ,  0.39742672,  0.05829744]))

In [20]:
# check that linear part matches
with np.printoptions(precision=5, suppress=True, linewidth=400):
    print(accel[3:])
    print(pindata.a_gf[1].linear)
    print(mjdata.qacc_smooth)
    print(mjdata.qfrc_smooth)

[-9.05989 -1.87551  1.91802]
[-0.12915  0.32349 -0.16417]
[-0.0428  -0.06305 -0.01692 -0.2789   0.39129  0.05578]
[-0.05883 -0.05083 -0.01213 -0.00865  0.00794  0.00396]


In [21]:
with np.printoptions(precision=5, suppress=True, linewidth=400):
    print(regressors.joint_body_regressor(mjmodel, mjdata, 1))

[[-9.05989 -0.6024   0.36044  0.84716  0.       0.       0.       0.       0.       0.     ]
 [-1.87551  0.47704 -0.9495   0.56095  0.       0.       0.       0.       0.       0.     ]
 [ 1.91802  0.05231  0.03992 -0.90656  0.       0.       0.       0.       0.       0.     ]
 [ 0.       0.       1.91802  1.87551 -0.26052 -0.05231 -0.30044  0.47704 -0.04294  0.30044]
 [ 0.      -1.91802  0.      -9.05989  0.44974  0.03992  0.39743 -0.30415 -0.36044 -0.44974]
 [ 0.      -1.87551  9.05989  0.      -0.41874  0.3471   0.41874 -0.56095  0.84716  0.0583 ]]


In [22]:
(
    np.allclose(regressors.joint_body_regressor(mjmodel, mjdata, 1), pinY),
    np.linalg.norm(regressors.joint_body_regressor(mjmodel, mjdata, 1) - pinY),
)

(False, 16.262269739443305)

In [23]:
velocity = np.zeros(6)
accel = np.zeros(6)
_cross = np.zeros(3)

mujoco.mj_objectVelocity(mjmodel, mjdata, 2, 1, velocity, 1)
mujoco.mj_rnePostConstraint(mjmodel, mjdata)
mujoco.mj_objectAcceleration(mjmodel, mjdata, 2, 1, accel, 1)

v, w = velocity[3:], velocity[:3]
# dv - classical acceleration, already contains g
dv, dw = accel[3:], accel[:3]
mujoco.mju_cross(_cross, w, v)

# dv -= _cross

v, w, dv, dw

(array([0.68314008, 0.72550602, 0.93695947]),
 array([0.79172504, 0.52889492, 0.56804456]),
 array([-9.05989038, -1.87550827,  1.91802153]),
 array([-0.2605178 ,  0.39742672,  0.05829744]))

In [24]:
print("Joint space forces")
print(f"Pinocchio: {pindata.f[1]} {result}")

rot = mjdata.xmat[1].reshape(3, 3)
print("rotation", rot)
mujtau = mjdata.qfrc_actuator.copy()

mujtau[:3] = rot.T @ mujtau[:3]
print(f"Mujoco: {mjdata.qfrc_actuator} {mujtau} {mjdata.qfrc_inverse}")

Joint space forces
Pinocchio:   f =  2.77556e-16  6.66134e-16 -8.32667e-17
tau = -3.46945e-17 -8.67362e-18  -4.0766e-17
 [ 2.77555756e-16  6.66133815e-16 -8.32667268e-17 -3.46944695e-17
 -8.67361738e-18 -4.07660017e-17]
rotation [[-0.12191288  0.06488634  0.99041759]
 [ 0.94387066  0.31621879  0.09546651]
 [-0.30699418  0.94646471 -0.09979545]]
Mujoco: [0. 0. 0. 0. 0. 0.] [0. 0. 0. 0. 0. 0.] [0. 0. 0. 0. 0. 0.]
